<h1 style="color:pink;">📊 SQL Proc uploading Script</h1>
<h3 style="color:purple;">This script involves: </h3>

        🔍 Detecting & Handling Excel File Formats
        🔐 Decrypting Encrypted Files
        📐 Analyzing Workbook Structures
        🔄 Consolidating Data into DataFrames

=========================================================================================================================

    Code Viewing Settings

In [ ]:
import pandas as pd
import csv

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_colwidth', None)
#pd.set_option('display.width', None)
#np.set_printoptions(threshold=np.inf)
#pd.set_option('display.float_format', lambda x: '%.5f' % x)

=========================================================================================================================

    Upload sql procs to sql server

In [ ]:
import pyodbc
import os

# SQL Server connection parameters
server = ''  # Replace with your SQL Server name
database = ''  # Replace with your database name
trusted_connection = 'yes'  # Use Windows authentication

# Directory containing the SQL procedure files
sql_proc_directory = r''

# Establish connection to SQL Server
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';Trusted_Connection=' + trusted_connection + ';')

try:
    cursor = conn.cursor()

    # Iterate over all .sql files in the directory
    for filename in os.listdir(sql_proc_directory):
        if filename.endswith('.sql'):
            filepath = os.path.join(sql_proc_directory, filename)
            with open(filepath, 'r') as file:
                sql_script = file.read()

                # Upload the stored procedure to the database without executing it
                cursor.execute(sql_script)
                print(f"Uploaded {filename} to {database}")

    # Commit changes
    conn.commit()
    print("All procedures uploaded successfully.")

except pyodbc.Error as e:
    print(f"Error uploading procedures: {str(e)}")

finally:
    # Close the connection
    conn.close()


In [ ]:
pip install networkx

In [ ]:
import pyodbc

# Database connection setup
server = ''
database = ''
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# Function to execute SQL query and return results
def execute_query(query):
    try:
        with pyodbc.connect(connection_string) as conn:
            cursor = conn.cursor()
            cursor.execute(query)
            results = cursor.fetchall()
            return results
    except Exception as e:
        print(f"Error: {e}")
        return []

# Query to get all stored procedures
query = """
    SELECT name
    FROM sys.procedures;
"""

# Fetch all stored procedures
results = execute_query(query)

# List all stored procedures
print("List of stored procedures:")
for row in results:
    print(row[0])


In [ ]:
import pyodbc
import re
import networkx as nx
import matplotlib.pyplot as plt

# Database connection setup
server = ''
database = ''
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# Function to execute SQL query and return results
def execute_query(query):
    try:
        with pyodbc.connect(connection_string) as conn:
            cursor = conn.cursor()
            cursor.execute(query)
            results = cursor.fetchall()
            return results
    except Exception as e:
        print(f"Error: {e}")
        return []

# Query to get all stored procedures and their definitions
query = """
    SELECT 
        p.name AS proc_name,
        m.definition AS proc_definition
    FROM 
        sys.sql_modules AS m
    INNER JOIN 
        sys.procedures AS p
        ON m.object_id = p.object_id;
"""

# Fetch all stored procedures and their definitions
results = execute_query(query)

# List all stored procedures
procedures = {}
for row in results:
    proc_name = row[0]
    proc_definition = row[1]
    procedures[proc_name] = proc_definition

# Function to find procedure calls within a procedure's definition
def find_proc_calls(proc_definition):
    pattern = re.compile(r'\bEXEC\b\s+(\w+)', re.IGNORECASE)
    return pattern.findall(proc_definition)

# Create a directed graph to represent the relationships
G = nx.DiGraph()

# Add each procedure as a node
for proc_name in procedures.keys():
    G.add_node(proc_name)

# Analyze each procedure to find calls to other procedures
for proc_name, proc_definition in procedures.items():
    called_procs = find_proc_calls(proc_definition)
    for called_proc in called_procs:
        if called_proc in procedures:  # Only consider calls to known procedures
            G.add_edge(proc_name, called_proc)

# Improved graph visualization
plt.figure(figsize=(14, 10))
pos = nx.spring_layout(G)  # Try using different layouts like circular_layout, shell_layout, kamada_kawai_layout

# Draw nodes
nx.draw_networkx_nodes(G, pos, node_size=3000, node_color='skyblue')

# Draw edges
nx.draw_networkx_edges(G, pos, edgelist=G.edges(), arrowstyle='->', arrowsize=20, edge_color='grey', width=2)

# Draw labels
nx.draw_networkx_labels(G, pos, font_size=12, font_family='sans-serif')

# Add a title
plt.title("Stored Procedures Relationship Graph")

# Remove axis
plt.axis('off')

# Show the plot
plt.show()


    The end